## Jupyter Notebook space for Passive Active NLP StageLens project

General module loading

In [1]:
!python --version
!python -m spacy download en_core_web_md

import spacy
import html
from spacy import displacy
import re
PATTERN = "[^a-zA-Z0-9_\s\.]+"
rgx = re.compile(PATTERN, re.IGNORECASE)

nlp = spacy.load('en_core_web_md')
print("Loaded models")

Python 3.6.3
    100% |████████████████████████████████| 120.9MB 34.4MB/s ta 0:00:01   43% |██████████████                  | 52.5MB 64.2MB/s eta 0:00:02

    Linking successful
    /srv/venv/lib/python3.6/site-packages/en_core_web_md -->
    /srv/venv/lib/python3.6/site-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')

Loaded models


Preprocessing input text

In [3]:
arr = [
["When people are helpless", "they need support to move forward. Helpless does not need to be a permanent descriptor, most people are able to be supported to redirect into a position which plays to their strengths."], 
["Technology", "has been one of the most significant disruptors of the very fabric of society and has been the capstone of hope for a ‘better future’ for many; and a cornerstone of deceit for a much larger engine of institutionalised social change - the ‘economy’, that has endlessly promised to make our lives simpler, easier, more comfortable, and more enjoyable. And when I take my mobile phone out of my pocket in the morning on the way to work to ring my wife, who is also on her way to work, technology is an amazing gift that allows two people who desire ongoing connection with each other the opportunity to do so."]
]

ips = []
for tok1, tok2 in arr:
    original_ip = "{} {}".format(tok1, tok2)
    ip = html.unescape(original_ip)
    ip = rgx.sub(' ', ip)
    ip = re.sub('\s+', ' ', ip)
    ips.append(ip)
    
print(ips)

['When people are helpless they need support to move forward. Helpless does not need to be a permanent descriptor most people are able to be supported to redirect into a position which plays to their strengths.', 'Technology has been one of the most significant disruptors of the very fabric of society and has been the capstone of hope for a better future for many and a cornerstone of deceit for a much larger engine of institutionalised social change the economy that has endlessly promised to make our lives simpler easier more comfortable and more enjoyable. And when I take my mobile phone out of my pocket in the morning on the way to work to ring my wife who is also on her way to work technology is an amazing gift that allows two people who desire ongoing connection with each other the opportunity to do so.']


In [3]:
print("<html><body>")
for ip in ips:
    doc = nlp(ip)
    a = displacy.render(doc, style='dep') # Saved in test.html
    print("<div>{}</div>".format(a))
print("</body></html>")

<html><body>
<div><svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" id="0" class="displacy" width="6350" height="574.5" style="max-width: none; height: 574.5px; color: #000000; background: #ffffff; font-family: Arial">
<text class="displacy-token" fill="currentColor" text-anchor="middle" y="484.5">
    <tspan class="displacy-word" fill="currentColor" x="50">When</tspan>
    <tspan class="displacy-tag" dy="2em" fill="currentColor" x="50">ADV</tspan>
</text>

<text class="displacy-token" fill="currentColor" text-anchor="middle" y="484.5">
    <tspan class="displacy-word" fill="currentColor" x="225">people</tspan>
    <tspan class="displacy-tag" dy="2em" fill="currentColor" x="225">NOUN</tspan>
</text>

<text class="displacy-token" fill="currentColor" text-anchor="middle" y="484.5">
    <tspan class="displacy-word" fill="currentColor" x="400">are</tspan>
    <tspan class="displacy-tag" dy="2em" fill="currentColor" x="400">VERB</tspan>
</text>

<text class=

<div><svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" id="0" class="displacy" width="19475" height="1449.5" style="max-width: none; height: 1449.5px; color: #000000; background: #ffffff; font-family: Arial">
<text class="displacy-token" fill="currentColor" text-anchor="middle" y="1359.5">
    <tspan class="displacy-word" fill="currentColor" x="50">Technology</tspan>
    <tspan class="displacy-tag" dy="2em" fill="currentColor" x="50">NOUN</tspan>
</text>

<text class="displacy-token" fill="currentColor" text-anchor="middle" y="1359.5">
    <tspan class="displacy-word" fill="currentColor" x="225">has</tspan>
    <tspan class="displacy-tag" dy="2em" fill="currentColor" x="225">VERB</tspan>
</text>

<text class="displacy-token" fill="currentColor" text-anchor="middle" y="1359.5">
    <tspan class="displacy-word" fill="currentColor" x="400">been</tspan>
    <tspan class="displacy-tag" dy="2em" fill="currentColor" x="400">VERB</tspan>
</text>

<text class="d

In [9]:
def customize_clause(arr, val, right = False):
    tval = val
    for x in arr:
        if tval.find(x)>=0 and x!=tval:
            tval = tval.replace(x, '')
    return tval
    
REQD_ATTRS = ('xcomp', 'ccomp', 'ROOT', 'advcl', 'relcl')
def split_clauses(doc):
    op = []
    for word in doc:
        if word.pos_ == "VERB":
            lefts = [l.text for l in word.lefts]
            rights = [r.text for r in word.rights]
            a = ' '.join(lefts + [word.text] + rights)
            #print(word.left_edge.i, word.right_edge.i, a)
            op.append(a)
    
    tdel = op.copy()
    op.sort()
    op.sort(key=len, reverse = True)
    new_op = ['' for x in op]
    for x in op:
        idx = tdel.index(x)
        new_text = customize_clause(op, x)
        new_op[idx] = new_text

    return new_op

def split_text(doc):
    op = []
    for sent in doc.sents:
        tm = split_clauses(sent)
        op = op + tm
    return op

for ip in ips:
    doc = nlp(ip)
    tt = split_text(doc)
    print("==================================================================================================")
    for x in tt:
        print(x)
        print()

When people are helpless 

they need support to move forward. 

Helpless does not need .

to be a permanent descriptor 

most people are able 

to be supported 

to redirect into a position 

which plays to their strengths

Technology has been one of the most significant disruptors of the very fabric of society and . 

has been the capstone of hope for a better future for many and a cornerstone of deceit for a much larger engine of institutionalised social change the economy 

that has endlessly promised 

to make 

our lives simpler easier more comfortable and more enjoyable

to work 

to ring my wife 

who is also on her way 

to work technology 

And when I take my mobile phone out of my pocket in the morning on the way is an amazing gift .

that allows two people 

who desire ongoing connection with each other the opportunity to do so

